In [2]:
import os
import tweepy 
from textblob import TextBlob

In [3]:
consumerKey="SkCfYErvpb3Crg2rkXWJEYMdb"
consumerSecret="aOmxf0jdockNnFh1psxUvBAt9ZjgBNL8llcDmlrkDnh0FaaIJY"
accessToken="796025544718487552-V9WZTwT1paHvhncDlqRbwBU3i5Ru21I"
accessTokenSecret="uTsAOvJW5X2lcrCPUohBfTtWztEDMloxOR9wpQBdiRXk5"

In [4]:
auth= tweepy.OAuthHandler(consumer_key=consumerKey,consumer_secret=consumerSecret)
auth.set_access_token(accessToken,accessTokenSecret)
api= tweepy.API(auth,wait_on_rate_limit=True)


In [5]:
searchTerm = '#coronavirus'
date_since = '2020-03-10'

In [6]:
tweets = tweepy.Cursor(api.search, q=searchTerm, lang="en", since=date_since).items(4000)

In [7]:
tweets

In [8]:
tweet_details = [[tweet.text,tweet.user.screen_name,TextBlob(tweet.text).sentiment.polarity] for tweet in tweets]

In [12]:
import pandas as pd

In [132]:
tweet_df = pd.DataFrame(data=tweet_details,columns=['text','username','sentiment'])

In [133]:
tweet_df.head(20)

,text,username,sentiment
0,RT @WAKhan337: It is a very sad news from Sri ...,Voice_Of_IYC,-0.583333
1,RT @IT_HealthPlus: How writing a daily journal...,Colaistebride,0.000000
2,RT @renoomokri: Imagine the reaction of \n\n* ...,aloyonyebuchi,0.050000
3,Indians returning from abroad will have to be ...,ManipalHealth,0.000000
4,RT @bizsupportgovuk: Get support to help with ...,boltoncouncil,0.000000
5,RT @RepSylviaGarcia: FREE #COVID19 testing at ...,Yourwinner3,0.400000
6,21 new cases and 1 new death in Norway \n\n[16...,COVIDLive,0.136364
7,RT @IGD_News: A man who's lived in the US for ...,treekisser,0.000000
8,RT @howardfineman: #Coronavirus: US death toll...,Boo824,0.000000
9,RT @TomFitton: The destructive shutdowns must ...,Unplugged_Neo,-0.325000


In [134]:
tweet_df['sentiment'] = tweet_df['sentiment'].apply(lambda x: 1 if x >0 else 0)

In [135]:
tweet_df['sentiment'].value_counts()

0    2471
1    1529
Name: sentiment, dtype: int64

In [136]:
tweet_df.head(20)

,text,username,sentiment
0,RT @WAKhan337: It is a very sad news from Sri ...,Voice_Of_IYC,0
1,RT @IT_HealthPlus: How writing a daily journal...,Colaistebride,0
2,RT @renoomokri: Imagine the reaction of \n\n* ...,aloyonyebuchi,1
3,Indians returning from abroad will have to be ...,ManipalHealth,0
4,RT @bizsupportgovuk: Get support to help with ...,boltoncouncil,0
5,RT @RepSylviaGarcia: FREE #COVID19 testing at ...,Yourwinner3,1
6,21 new cases and 1 new death in Norway \n\n[16...,COVIDLive,1
7,RT @IGD_News: A man who's lived in the US for ...,treekisser,0
8,RT @howardfineman: #Coronavirus: US death toll...,Boo824,0
9,RT @TomFitton: The destructive shutdowns must ...,Unplugged_Neo,0


In [137]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding, Activation
from tensorflow.keras.layers import Conv1D , MaxPooling1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras import regularizers
import numpy as np
from numpy import array
from sklearn.model_selection import train_test_split

In [138]:
text = tweet_df['text'].tolist()
text

['RT @WAKhan337: It is a very sad news from Sri Lanka that the dead body of Muslims is being treated inhumanely &amp; their body is not being all…',
 'RT @IT_HealthPlus: How writing a daily journal changes my mood. \n#Coronavirus \nhttps://t.co/o1tKAj84AH',
 'RT @renoomokri: Imagine the reaction of \n\n* General @MBuhari\n* @Elrufai\n* @ObyEzeks\n* Wole Soyinka\n* @AsiwajuTinubu\n\nIf @GEJonathan was Pre…',
 'Indians returning from abroad will have to be quarantined at home or at medical facilities to ensure that the virus… https://t.co/tO3k4cSTJI',
 'RT @bizsupportgovuk: Get support to help with the impact of #coronavirus on your business:\n\n✅ loans, tax relief and cash grants \n✅ keeping…',
 "RT @RepSylviaGarcia: FREE #COVID19 testing at @WeAreHCCS Southeast at 6815 Rustic. You don't need to have symptoms or make an appointment!…",
 '21 new cases and 1 new death in Norway \n\n[16:47 GMT] #coronavirus #CoronaVirusUpdate #COVID19 #CoronavirusPandemic',
 "RT @IGD_News: A man who's live

In [139]:
y=tweet_df['sentiment']

In [140]:
token = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n🙏🚨◘📣⚠️🌏🤔✅☣️️⭐',lower=True, split=" ")
token.fit_on_texts(text)

In [141]:
token

In [142]:
vocab_size = len(token.word_index)+1
vocab_size

12066

In [143]:
print(token.index_word)

{1: 'the', 2: 'rt', 3: 'coronavirus', 4: 'to', 5: 'https', 6: 'co', 7: 't', 8: 'of', 9: 'in', 10: 'and', 11: 'a', 12: 'is', 13: 'for', 14: 'on', 15: 'covid19', 16: 'are', 17: 'this', 18: 'as', 19: 'be', 20: 'with', 21: 'you', 22: 'new', 23: 'from', 24: 'that', 25: '…', 26: 'we', 27: 'cases', 28: 'at', 29: 'have', 30: 'amp', 31: 'will', 32: 'covid', 33: 'i', 34: 'has', 35: 'people', 36: '19', 37: 'deaths', 38: 'it', 39: 'not', 40: 'us', 41: 'pandemic', 42: 'by', 43: 'more', 44: 'but', 45: 'if', 46: 'who', 47: 'total', 48: 'our', 49: 'can', 50: 'an', 51: 'about', 52: 'your', 53: 'no', 54: 'how', 55: 'during', 56: 'they', 57: '1', 58: 'lockdown', 59: 'my', 60: 'trump', 61: 'all', 62: 'what', 63: 'out', 64: 'now', 65: 'was', 66: 'been', 67: 'here', 68: 'may', 69: 'watch', 70: 'he', 71: 'health', 72: 'their', 73: 'do', 74: 'like', 75: 'due', 76: 'after', 77: 'or', 78: 'tomfitton', 79: '000', 80: 'up', 81: 'just', 82: 'today', 83: 'one', 84: 'first', 85: 'there', 86: 'home', 87: 'giving', 88

In [144]:
x=['my health is fine']

In [145]:
token.texts_to_sequences(x)

[[59, 71, 12]]

In [146]:
encoded_text = token.texts_to_sequences(text)

In [147]:
print(encoded_text)

[[2, 5243, 38, 12, 11, 487, 877, 106, 23, 1632, 3525, 24, 1, 695, 428, 8, 1451, 12, 167, 1889, 3526, 30, 72, 428, 12, 39, 167, 1284], [2, 38, 5244, 54, 3527, 11, 100, 1890, 1035, 59, 2243, 3, 5, 7, 6, 5245], [2, 1036, 1168, 1, 1633, 8, 418, 1634, 1452, 1635, 1636, 1637, 1638, 45, 1639, 65, 1453], [2244, 2245, 23, 5246, 31, 29, 4, 19, 5247, 28, 86, 77, 28, 321, 1640, 4, 1285, 24, 1, 3528, 5, 7, 6, 5248], [2, 3529, 113, 201, 4, 95, 20, 1, 347, 8, 3, 14, 52, 488, 3530, 1286, 489, 10, 1454, 3531, 3532], [2, 2246, 397, 15, 195, 28, 1891, 1892, 28, 1893, 1894, 21, 398, 137, 4, 29, 1287, 77, 219, 50, 1037, 25], [646, 22, 27, 10, 57, 22, 129, 9, 2738, 807, 1641, 1455, 3, 1288, 15, 878], [2, 2739, 106, 11, 808, 2740, 2741, 9, 1, 40, 13, 809, 461, 12, 2247, 80, 42, 1289, 2742, 160, 4, 301, 8, 32, 36, 9, 2248, 2743, 419, 2744], [2, 513, 3, 40, 129, 202, 173, 29, 66, 490, 133, 38, 462, 151, 250, 491, 281, 90, 5, 7, 6, 492], [2, 78, 1, 220, 223, 92, 142, 179, 203, 16, 196, 1, 211, 4, 224, 103, 10, 

In [148]:
max_length = 120
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')

In [149]:
print(X)

[[    2  5243    38 ...     0     0     0]
 [    2    38  5244 ...     0     0     0]
 [    2  1036  1168 ...     0     0     0]
 ...
 [    2 12050 12051 ...     0     0     0]
 [    2 12057    17 ...     0     0     0]
 [    2  5028   102 ...     0     0     0]]


In [150]:
X.shape

(4000, 120)

In [151]:
X = np.array(X)
y = np.array(y)

In [152]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.2, stratify=y)

In [153]:
vec_size = 300

model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length = max_length))

model.add(Conv1D(64, 8, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(16, activation='relu'))

model.add(GlobalMaxPooling1D())

model.add(Dense(1,activation='sigmoid'))

In [154]:
model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])

In [155]:
%%time
model.fit(X_train, y_train, epochs = 5, validation_data= (X_test, y_test))

Train on 3200 samples, validate on 800 samples
Epoch 1/5
3200/3200 [==============================] - 54s 17ms/sample - loss: 0.6335 - accuracy: 0.6319 - val_loss: 0.5213 - val_accuracy: 0.7625
Epoch 2/5
3200/3200 [==============================] - 17s 5ms/sample - loss: 0.3263 - accuracy: 0.8625 - val_loss: 0.2730 - val_accuracy: 0.8863
Epoch 3/5
3200/3200 [==============================] - 17s 5ms/sample - loss: 0.0491 - accuracy: 0.9875 - val_loss: 0.2757 - val_accuracy: 0.8800
Epoch 4/5
3200/3200 [==============================] - 17s 5ms/sample - loss: 0.0131 - accuracy: 0.9975 - val_loss: 0.2732 - val_accuracy: 0.8963
Epoch 5/5
3200/3200 [==============================] - 16s 5ms/sample - loss: 0.0043 - accuracy: 0.9997 - val_loss: 0.2893 - val_accuracy: 0.8825
Wall time: 2min 4s


In [156]:
def get_encoded(x):
    x= token.texts_to_sequences(x)
    x=pad_sequences(x, maxlen=max_length, padding = 'post')
    return x

In [157]:
x = ['worst services. will not come again']

In [158]:
model.predict_classes(get_encoded(x))

array([[0]])

In [159]:
x = ['this pandemic is not going to end soon. please stay home and be safe ']

In [160]:
model.predict_classes(get_encoded(x))

array([[0]])

In [161]:
x = ['this pandemic is making me crazy i cannot take this anymore ']

In [162]:
model.predict_classes(get_encoded(x))

array([[0]])

In [163]:
x = ['loving this lockdown . i have done so many courses and learned a lot']

In [164]:
model.predict_classes(get_encoded(x))

array([[1]])

In [165]:
x = ['corona virus has made our lifes better']

In [166]:
model.predict_classes(get_encoded(x))

array([[1]])

In [167]:
x = ['i will never say lies again']

In [168]:
model.predict_classes(get_encoded(x))

array([[0]])

In [169]:
x = ['corona is fake']

In [170]:
model.predict_classes(get_encoded(x))

array([[0]])

In [204]:
from pathlib import Path
emb_dict = {}
glove = open(r'C:\Users\Surbhi\Desktop\New folder\glove.twitter.27B.50d.txt', encoding="utf8")
for line in glove:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    emb_dict[word] = vector
glove.close()

In [206]:
tem = ['positive', 'lockdown', 'corona', 'stay']
for w in tem:
    if w in emb_dict.keys():
        print('Found the word {} in the dictionary'.format(w))

Found the word positive in the dictionary
Found the word lockdown in the dictionary
Found the word corona in the dictionary
Found the word stay in the dictionary


In [208]:
emb_matrix = np.zeros((vocab_size, 50))

for w, i in token.word_index.items():
    # The word_index contains a token for all words of the training data so we need to limit that
    if i < vocab_size:
        vect = emb_dict.get(w)
        # Check if the word from the training data occurs in the GloVe word embeddings
        # Otherwise the vector is kept with only zeros
        if vect is not None:
            emb_matrix[i] = vect
    else:
        break

In [211]:
glove_model = Sequential()
glove_model.add(Embedding(vocab_size,50, input_length=max_length))
glove_model.add(Flatten())
glove_model.add(Dense(1, activation='sigmoid'))
glove_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 50)           603300    
_________________________________________________________________
flatten (Flatten)            (None, 6000)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 6001      
Total params: 609,301
Trainable params: 609,301
Non-trainable params: 0
_________________________________________________________________


In [212]:
glove_model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])

In [213]:
%%time
glove_model.fit(X_train, y_train, epochs = 5, validation_data= (X_test, y_test))

Train on 3200 samples, validate on 800 samples
Epoch 1/5
3200/3200 [==============================] - 33s 10ms/sample - loss: 0.6217 - accuracy: 0.6441 - val_loss: 0.5608 - val_accuracy: 0.7412
Epoch 2/5
3200/3200 [==============================] - 2s 763us/sample - loss: 0.4017 - accuracy: 0.8634 - val_loss: 0.4186 - val_accuracy: 0.7912
Epoch 3/5
3200/3200 [==============================] - 2s 736us/sample - loss: 0.1899 - accuracy: 0.9825 - val_loss: 0.3600 - val_accuracy: 0.8238
Epoch 4/5
3200/3200 [==============================] - 2s 633us/sample - loss: 0.0817 - accuracy: 0.9975 - val_loss: 0.3391 - val_accuracy: 0.8275
Epoch 5/5
3200/3200 [==============================] - 2s 645us/sample - loss: 0.0394 - accuracy: 0.9997 - val_loss: 0.3321 - val_accuracy: 0.8338
Wall time: 45.5 s


In [214]:
x = ['worst services. will not come again']
model.predict_classes(get_encoded(x))

array([[0]])

In [215]:
x = ['worst services. will not come again']
glove_model.predict_classes(get_encoded(x))

array([[0]])

In [217]:
x = ['i will never say lies again']
glove_model.predict_classes(get_encoded(x))

array([[0]])

In [221]:
x = ['Instead of dumping thousands of liters of beer that could not be delivered due to #coronavirus closures, a brewery in Germany got creative — and is giving it away for free.']
glove_model.predict_classes(get_encoded(x))

array([[0]])

In [223]:
x = ['How interesting! S Korea actually didn’t do much Testing! Because they started Testing IMMEDIATELY at the Beginning, they caught it & stopped #Corona Early like a Fire. Only 248 deaths!! TRUMP took FOREVER, let the Fire SPREAD without doing anything, & now it burned everything!']
glove_model.predict_classes(get_encoded(x))

array([[0]])

In [224]:
x = ['what the hell is this']
glove_model.predict_classes(get_encoded(x))

array([[0]])

In [225]:
x = ['Congratulations Delhi. You voted for freebies.  All zones in Delhi are red zones.  So enjoy free electricity and water in your locked houses Delhi is that part of India where neither CM nor HMO bothered']
glove_model.predict_classes(get_encoded(x))

array([[1]])